<a href="https://colab.research.google.com/github/prachikashikar/ExpoShare/blob/main/Copy_of_Other_model_bfloat16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Source : https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms 
import numpy as np
import time
from torchvision import models


In [2]:
model = torchvision.models.shufflenet_v2_x0_5()


In [ ]:
New_PATH = 'mnasnet0_5.pt'
torch.save(model.state_dict(), New_PATH)

In [8]:
New_PATH = 'shufflenet_v2_x0_5_retrained.pt'
model.load_state_dict(torch.load(New_PATH))


<All keys matched successfully>

In [3]:
# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
# only for colab

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
#only for Colab

!unzip gdrive/My\ Drive/d_trainData.zip

Streaming output truncated to the last 5000 lines.
  inflating: trainData/938_cauliflower/026_3046.jpg  
  inflating: trainData/938_cauliflower/028_8994.jpg  
  inflating: trainData/938_cauliflower/029_4632.jpg  
  inflating: trainData/938_cauliflower/030_6124.jpg  
  inflating: trainData/938_cauliflower/032_5546.jpg  
  inflating: trainData/938_cauliflower/033_7223.jpg  
  inflating: trainData/938_cauliflower/034_626.jpg  
  inflating: trainData/938_cauliflower/035_9103.jpg  
  inflating: trainData/938_cauliflower/036_8284.jpg  
  inflating: trainData/938_cauliflower/037_2735.jpg  
  inflating: trainData/938_cauliflower/038_3613.jpg  
  inflating: trainData/938_cauliflower/039_2623.jpg  
  inflating: trainData/938_cauliflower/040_7333.jpg  
  inflating: trainData/938_cauliflower/041_7874.jpg  
  inflating: trainData/938_cauliflower/042_4176.jpg  
  inflating: trainData/938_cauliflower/044_2364.jpg  
  inflating: trainData/938_cauliflower/045_8720.jpg  
  inflating: trainData/938_cauli

In [6]:
#only for Colab

!unzip gdrive/My\ Drive/d_testData.zip

Streaming output truncated to the last 5000 lines.
  inflating: testData/610_jersey, T-shirt, tee shirt/093_8952.jpg  
  inflating: testData/610_jersey, T-shirt, tee shirt/095_8441.jpg  
   creating: testData/320_damselfly/
  inflating: testData/320_damselfly/009_7605.jpg  
  inflating: testData/320_damselfly/021_9054.jpg  
  inflating: testData/320_damselfly/022_1467.jpg  
  inflating: testData/320_damselfly/027_1699.jpg  
  inflating: testData/320_damselfly/031_3417.jpg  
  inflating: testData/320_damselfly/035_9404.jpg  
  inflating: testData/320_damselfly/036_3643.jpg  
  inflating: testData/320_damselfly/044_2751.jpg  
  inflating: testData/320_damselfly/046_130.jpg  
  inflating: testData/320_damselfly/048_7366.jpg  
  inflating: testData/320_damselfly/054_1100.jpg  
  inflating: testData/320_damselfly/056_142.jpg  
  inflating: testData/320_damselfly/059_1671.jpg  
  inflating: testData/320_damselfly/064_9708.jpg  
  inflating: testData/320_damselfly/068_7226.jpg  
  inflating: 

In [ ]:
# Source (https://github.com/tjmoon0104/pytorch-tiny-imagenet) and 
#(https://towardsdatascience.com/pytorch-ignite-classifying-tiny-imagenet-with-efficientnet-e5b1768e5e8f)

from torchvision import datasets
from torch.utils.data import DataLoader

batch_size = 64
# Setup function to create dataloaders for image datasets
def generate_dataloader(data, name, transform):
    if data is None: 
        return None
    
    
    
    # Read image files to pytorch dataset using ImageFolder, a generic data 
    # loader where images are in format root/label/filename
    # See https://pytorch.org/vision/stable/datasets.html
    if transform is None:
        dataset = datasets.ImageFolder(data, transform=T.ToTensor())
    else:
        dataset = datasets.ImageFolder(data, transform=transform)

    # Set options for device
    '''
    if use_cuda:
        kwargs = {"pin_memory": True, "num_workers": 1}
    else:
        kwargs = {}
    '''
    kwargs = {}
    print(dataset)
    # Wrap image dataset (defined above) in dataloader 
    dataloader = DataLoader(dataset, batch_size=batch_size, 
                        shuffle=True)
            
    
    return dataloader

In [9]:
transform = transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

my_testDataLoader = generate_dataloader("testData", "test",
                                  transform=transform)

# train_loader is from validation set
my_train_loader = generate_dataloader("trainData", "train",
                                  transform=transform)

Dataset ImageFolder
    Number of datapoints: 20000
    Root location: testData
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImageFolder
    Number of datapoints: 80001
    Root location: trainData
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [ ]:
# Float32 Testing

correct_5 = 0
correct = 0
total = 0
c=0
model.float()
start_epoch = time.perf_counter_ns()
for data in my_testDataLoader:
    c=c+1
    
    images, labels = data
    outputs = model(images)
    # the class with the highest energy is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)    
   
    correct += (predicted == labels).sum().item()
    pred = outputs.topk(5,1,largest=True,sorted=True)[1]
    lab=labels.view(labels.size(0),-1).expand_as(pred)
    c=pred.eq(lab).float()
    correct_5+=c[:,:5].sum()
    print(correct_5)
    print(correct)
    
stop_epoch = time.perf_counter_ns()
print('Accuracy of the network on the test images: %f %%' % (100 * correct / total))    
print('Top 5 Accuracy of the network on the test images: %d %%' % (100 * correct_5 / total)) 
print('Execution time in ns:', stop_epoch-start_epoch)
    

tensor(52.)
39
tensor(106.)
82
tensor(160.)
125
tensor(214.)
161
tensor(270.)
203
tensor(324.)
244


KeyboardInterrupt: ignored

In [10]:
# The original Memory requirement

global_original_mem = 0
global_bfloat16_mem = 0
for name, param in model.named_parameters():
  if 'bias' in name:
    continue
  x = param.data
  global_original_mem = global_original_mem + 32*torch.numel(x)
  global_bfloat16_mem = global_bfloat16_mem + 16*torch.numel(x)

print("Original Memory and bfloatt16 requirement is : ",global_original_mem,global_bfloat16_mem," bits")    

Original Memory and bfloatt16 requirement is :  43578112 21789056  bits


In [ ]:
# Exponent Sharing on Inference

global_mem_after_compression = 0    
global_mem_after_compression_bfloat16 = 0    

for name, param in model.named_parameters():
    if 'bias' in name:
        continue
    print("................................................................")
    print(name)
    x = param.data    
    mant,exp = x.frexp()
    #print(exp.dtype)
    expo = exp.numpy()
    xNumpy = x.numpy()
    
    # Plot bar graph
    unique, counts = np.unique(exp, return_counts=True)
    print("Original Exponents,",unique)
    print('Unique exponents from original are = ',len(unique))
    Bits_required = np.ceil(np.log2(len(unique)))    
    
    
    global_mem_after_compression = global_mem_after_compression + torch.numel(x) * (23 + Bits_required+1 )+ 8*len(unique)
    global_mem_after_compression_bfloat16 = global_mem_after_compression_bfloat16 + torch.numel(x) * (7 + Bits_required+1 )+ 8*len(unique)
    #print("before after",16*torch.numel(x) ,torch.numel(torch.from_numpy(new_x)) * (7 + index_bits_for_top_exponents+1 )+ 8*len(top_exponents) )
    print("")
    #break  

print ('Memory saved in for this model is :',100* (global_original_mem-global_mem_after_compression)/global_original_mem,'%') 
print ('Bfloat16: Memory saved in for this model is :',100* (global_bfloat16_mem-global_mem_after_compression_bfloat16)/global_bfloat16_mem,'%')    

................................................................
layers.0.weight
Original Exponents, [-8 -7 -6 -5 -4 -3 -2 -1  0  1  2  3  4  5]
Unique exponents from original are =  14

................................................................
layers.1.weight
Original Exponents, [1 2 3 4]
Unique exponents from original are =  4

................................................................
layers.3.weight
Original Exponents, [-7 -4 -3 -2 -1  0  1  2  3  4  5]
Unique exponents from original are =  11

................................................................
layers.4.weight
Original Exponents, [0 1 2 3 4]
Unique exponents from original are =  5

................................................................
layers.6.weight
Original Exponents, [-6 -5 -4 -3 -2 -1  0  1  2  3  4  5]
Unique exponents from original are =  12

................................................................
layers.7.weight
Original Exponents, [3 4 5]
Unique exponents from original are =  3

TypeError: ignored

In [12]:
# Find distict exponent in whole model and new expo table with 1 less ind bit

expTable = torch.empty(0)
expTable = expTable.to(device)
print('expTable',expTable.device)

model.float()
model = model.to(device)
for name, param in model.named_parameters():
    print("................................................................")
    print(name)
    if 'bias' in name:
        continue
    x = param.data
    print(x.dtype)
    mant, exp = torch.frexp(x)
    expC, count = exp.unique(return_counts=True)
    exp = exp.ravel()
    print('This layer',exp.unique())
    expTable = torch.cat((expTable,exp),-1) 
    
    print('Added expes',expTable.unique())
    
    
    #break
expTable = expTable.unique() 
expTable, ind = expTable.sort(descending = True)
print("Orig ExpTable:",expTable)
print(len(expTable))
orgIndBits = np.ceil(np.log2(len(expTable)))
print(orgIndBits)
newIndBits = orgIndBits - 1
newDistExpCount = np.power(2,newIndBits)
print("New total exponenets: ",newDistExpCount)
newExpTable = expTable[:newDistExpCount.astype(int)] 
print("New exp Table: ",newExpTable)

expTable cuda:0
................................................................
conv1.0.weight
torch.float32
This layer tensor([-9, -8, -7, -6, -5, -4, -3, -2, -1,  0,  1,  2], device='cuda:0',
       dtype=torch.int32)
Added expes tensor([-9., -8., -7., -6., -5., -4., -3., -2., -1.,  0.,  1.,  2.],
       device='cuda:0')
................................................................
conv1.1.weight
torch.float32
This layer tensor([-9, -8, -1,  0,  1], device='cuda:0', dtype=torch.int32)
Added expes tensor([-9., -8., -7., -6., -5., -4., -3., -2., -1.,  0.,  1.,  2.],
       device='cuda:0')
................................................................
conv1.1.bias
................................................................
stage2.0.branch1.0.weight
torch.float32
This layer tensor([-6, -5, -4, -3, -2, -1,  0,  1,  2,  3], device='cuda:0',
       dtype=torch.int32)
Added expes tensor([-9., -8., -7., -6., -5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.],
       device='cuda:0')
...

In [11]:
batch_size = 64
num_classes = 1000
learning_rate = 0.001 
#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#this is defined to print how many steps are remaining when training
total_step = len(my_train_loader)


In [21]:
# Fine tuning
# Autocast Ref : https://pytorch.org/tutorials/recipes/recipes/amp_recipe.html
# Find all masks


num_epochs = 3
ep = 1
newExpTable = newExpTable.to(device)  

model.to('cuda')
for epoch in range(num_epochs):
    print("starting epoch", ep)

    ep = ep+1
    for i, (images, labels) in enumerate(my_train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward pass
        model.float()
        outputs = model(images)
        loss = cost(outputs, labels)
        	
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('i is',i)
        # Alter the weight update by exponent value
        for name, param in model.named_parameters():
            if 'bias' in name:
                continue
            #print("layer: ",name)
            x = param.data
            mant,exp = torch.frexp(x) 
           
            # pytorch option to np.vectorize
            exp_1 = exp.flatten()
        
            mask_2 = torch.zeros(exp_1.shape)
            for e in range(exp_1.size(0)):
              if exp_1[e] not in newExpTable:
                mask_2[e] = 1           
            mask_2 = mask_2.to(device)                   
            mask_2 = mask_2.reshape(exp.shape) 
            print('newExpTable exp_1',newExpTable.shape,exp_1.shape)  
            newExpTable = newExpTable.expand_as(exp_1)
            mask = torch.where(exp_1 in newExpTable ,0,1)   
            mask = mask.reshape(exp.shape)   

            if(mask == mask_2):
              print('success')
              break        
           
            parm_to_be_adjusted = torch.mul(mask_2,x)
            params_with_qualified_expo = x-parm_to_be_adjusted    
            flattened_search_params = params_with_qualified_expo.flatten()
                     
            parm_to_be_adjusted = parm_to_be_adjusted.flatten()
            changed_params = torch.zeros(parm_to_be_adjusted.shape).to(device) 
            for p in range(parm_to_be_adjusted.size(0)):
              if parm_to_be_adjusted[p] !=0:
                val = parm_to_be_adjusted[p]
                diff = (flattened_search_params - val)
                diff = torch.abs(diff)
                min_ind = diff.argmin()
                changed_params[p] = flattened_search_params[min_ind] 

                    
            changed_params = changed_params.reshape(params_with_qualified_expo.shape)
            updated_weights = torch.add(params_with_qualified_expo, changed_params)
            param.data = updated_weights           
                 
        		
        if (i+1) % 40 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}...................................' 
        		           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    pathstring = 'shufflenet_v2_x0_5_retrained_' + str(ep)+'.pt'
    print('New_PATH is ', pathstring)
    model.load_state_dict(torch.load(pathstring))
            
            

starting epoch 1
i is 0
newExpTable exp_1 torch.Size([8]) torch.Size([648])


RuntimeError: ignored

In [ ]:
# Fine tuning using masks

for epoch in range(num_epochs):
    print("starting epoch", ep)

    ep = ep+1
    for i, (images, labels) in enumerate(my_train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward pass
        model.float()
        outputs = model(images)
        loss = cost(outputs, labels)
        	
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('i is',i)
        # Alter the weight update by exponent value
        for name, param in model.named_parameters():
            if 'bias' in name:
                continue
            #print("layer: ",name)
            x = param.data
            mask = 
            parm_to_be_adjusted = torch.mul(mask_2,x)
            params_with_qualified_expo = x-parm_to_be_adjusted    
            flattened_search_params = params_with_qualified_expo.flatten()
                     
            parm_to_be_adjusted = parm_to_be_adjusted.flatten()
            changed_params = torch.zeros(parm_to_be_adjusted.shape).to(device) 
            for p in range(parm_to_be_adjusted.size(0)):
              if parm_to_be_adjusted[p] !=0:
                val = parm_to_be_adjusted[p]
                diff = (flattened_search_params - val)
                diff = torch.abs(diff)
                min_ind = diff.argmin()
                changed_params[p] = flattened_search_params[min_ind] 

                    
            changed_params = changed_params.reshape(params_with_qualified_expo.shape)
            updated_weights = torch.add(params_with_qualified_expo, changed_params)
            param.data = updated_weights           
                 
        		
        if (i+1) % 40 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}...................................' 
        		           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    pathstring = 'shufflenet_v2_x0_5_retrained_' + str(ep)+'.pt'
    print('New_PATH is ', pathstring)
    model.load_state_dict(torch.load(pathstring))
            

In [ ]:
New_PATH = 'mnasnet0_5_retrained.pt'
torch.save(model.state_dict(), New_PATH)

In [ ]:
New_PATH = 'mnasnet0_5_retrained_1.pt'
model.load_state_dict(torch.load(New_PATH))
model

MNASNet(
  (layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (7): BatchNorm2d(8, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
    (8): Sequential(
      (0): _InvertedResidual(
        (layers): Sequential(
          (0): Conv2d(8, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.00029999999999996696, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(24, 24,

# New Section

# New Section

In [ ]:
# Functions to change the storage formats

def retrive_the_flots(mant, ind, expList):    
    
    print(type(mant), type(ind), type(expList))
    in_line_func = lambda x:expList[x]
    vectorized_func = np.vectorize(in_line_func)
    #exp_tensor = vectorized_func(ind)    
    return torch.ldexp(mant,torch.from_numpy(vectorized_func(ind)))  


def convertToNewFormat(x):
    mant, exp = torch.frexp(x)
    exp_table = exp.unique()
    exp_table = exp_table.tolist()
    in_line_func = lambda x:exp_table.index(x)
    vectorized_func = np.vectorize(in_line_func)
    index_tensor = vectorized_func(exp) 
    index_tensor= torch.from_numpy(index_tensor)
    exp_table = np.array(exp_table)
    exp_table = torch.from_numpy(exp_table)
    return mant,index_tensor,exp_table

In [ ]:
# Execution overhead calculation

base_time = 0
expo_share_time = 0
for name, param in model.named_parameters():    
    if 'bias' in name:
        continue
    if '1.weight' in name:
        continue
    print(name) 
    
    x = param.data
    start_epoch = time.perf_counter_ns()
    y = x
    end_epoch = time.perf_counter_ns()
    base_time = base_time + (end_epoch - start_epoch)

for name, param in model.named_parameters():    
    if 'bias' in name:
        continue
    if '1.weight' in name:
        continue
    print(name)   
    x = param.data
    mant,ind,expTable = convertToNewFormat(x)
    start_epoch = time.perf_counter_ns()
    y = retrive_the_flots(mant,ind,expTable)
    end_epoch = time.perf_counter_ns()
    expo_share_time = expo_share_time + (end_epoch - start_epoch)
    

print("Base time in ns: ",base_time)
print("Time after exponent sharing in ns: ",expo_share_time)

In [ ]:
# Exponent Approximation and Sharing on Inference (independent cell)

bitsToBeRemoved = 1

global_mem_after_compression = 0    
global_mem_after_compression_bfloat16 = 0    

for name, param in model.named_parameters():
    print("................................................................")
    print(name)
    if 'bias' in name:
        continue
    x = param.data    
    mant,exp = x.frexp()
    #print(exp.dtype)
    expo = exp.numpy()
    xNumpy = x.numpy()
    
    # Plot bar graph
    unique, counts = np.unique(exp, return_counts=True)
    print("Original Exponents,",unique)
    print('Unique exponents from original are = ',len(unique))
    Bits_required = np.ceil(np.log2(len(unique)))
    unique[::-1].sort()
    if(Bits_required ==1):
        global_mem_after_compression = global_mem_after_compression + torch.numel(x) * (23 + index_bits_for_top_exponents+1 )+ 8*len(top_exponents)
        global_mem_after_compression_bfloat16 = global_mem_after_compression_bfloat16 + torch.numel(x) * (7 + index_bits_for_top_exponents+1 )+ 8*len(top_exponents)
        print("before after",16*torch.numel(x) ,torch.numel(torch.from_numpy(new_x)) * (7 + index_bits_for_top_exponents+1 )+ 8*len(top_exponents) )
        print("No further compression possible in this layer" )
        continue
    total_expo_after_pruning = np.power(2,(Bits_required- bitsToBeRemoved))
    #print("Top expo with high magnetudes",total_expo_after_pruning.dtype)
    top_exponents = unique[:total_expo_after_pruning.astype(int)]
    print("Expo selected after pruning",top_exponents)
    #smallest_expo = top_exponents[total_expo_after_pruning.astype(np.int64) - 1]
    
    
    index_bits_for_top_exponents = Bits_required - bitsToBeRemoved
    
       
     # Remove to_be_changed_params and make new tensor
    in_line_func_1 = lambda x:1 if x in top_exponents else 0
    vectorized_func = np.vectorize(in_line_func_1)
    mask = vectorized_func(expo)
    new_x = np.multiply(xNumpy,mask)
    
    to_be_changed_params = x - new_x
   
    new_x_flat = torch.from_numpy(new_x)
    new_x_flat = new_x_flat.flatten()
    
    changed_params = to_be_changed_params
    for p in np.ndindex(to_be_changed_params.shape):
        val = to_be_changed_params[p]
        if val==0:
            continue
        else:
            #print(p)
            diff = np.abs(new_x_flat - val)
            min_ind = diff.argmin()
            changed_params[p] = new_x_flat[min_ind] 
    updated_weights = np.add(new_x, changed_params)
    
    param.data = updated_weights
    
    
    global_mem_after_compression = global_mem_after_compression + torch.numel(torch.from_numpy(new_x)) * (23 + index_bits_for_top_exponents+1 )+ 8*len(top_exponents)
    global_mem_after_compression_bfloat16 = global_mem_after_compression_bfloat16 + torch.numel(torch.from_numpy(new_x)) * (7 + index_bits_for_top_exponents+1 )+ 8*len(top_exponents)
    #print("before after",16*torch.numel(x) ,torch.numel(torch.from_numpy(new_x)) * (7 + index_bits_for_top_exponents+1 )+ 8*len(top_exponents) )
    print("")
    #break  

print ('Memory saved in for this model is :',100* (global_original_mem-global_mem_after_compression)/global_original_mem,'%') 
print ('Bfloat16: Memory saved in for this model is :',100* (global_bfloat16_mem-global_mem_after_compression_bfloat16)/global_bfloat16_mem,'%')    
   